#  GBDT算法的核心思想

GBDT是Gradient Boosting Decision Tree（梯度提升树）的缩写。GBDT算法也是一种非常实用的Boosting算法，它与AdaBoost算法的区别在于：AdaBoost算法根据分类效果调整权重并不断迭代，最终生成强学习器；GBDT算法则将损失函数的负梯度作为残差的近似值，不断使用残差迭代和拟合回归树，最终生成强学习器。简单来说，AdaBoost算法是调整权重，而GBDT算法则是拟合残差。

# GBDT算法的简单代码实现
GBDT算法既能做分类分析，又能做回归分析，对应的模型分别为GBDT分类模型（GradientBoostingClassifier）和GBDT回归模型（GradientBoostingRegressor）。GBDT分类模型的弱学习器是分类决策树模型，GBDT回归模型的弱学习器则是回归决策树模型。

### 1、GBDT分类模型

In [5]:
from sklearn.ensemble import GradientBoostingClassifier 
x = [[1,2], [3,4], [5,6], [7,8], [9,10]] # X是特征变量，共有2个特征
y = [0, 0, 0, 1, 1] # y是目标变量，共有2个分类——0和1

model = GradientBoostingClassifier(random_state=1) # 设置random_state参数为1（该数字没有特殊含义，可以换成其他数字），使得每次运行结果保持一致
model.fit(x, y) # 用fit()函数训练模型

print(model.predict([[5, 5]])) # 用predict()函数进行预测

[0]


### 2、GBDT回归模型

In [8]:
from sklearn.ensemble import GradientBoostingRegressor
x = [[1,2], [3,4], [5,6], [7,8], [9,10]] # X是特征变量，共有2个特征
y = [1, 2, 3, 4, 5] # y是目标变量，它是一个连续值

model = GradientBoostingRegressor(random_state=1) # 设置random_state参数为1（该数字没有特殊含义，可以换成其他数字），使得每次运行结果保持一致
model.fit(x, y) # 用fit()函数训练模型

print(model.predict([[5, 5]])) # 用predict()函数进行预测

[2.44204599]


# 案例：产品定价模型
出版社在对图书进行定价时会考虑图书的页数、纸张、类别、内容、作者及读者等很多因素，用人工来分析较为烦琐，并且容易遗漏。如果能建立一个模型综合考虑各方面因素对图书进行定价，那么就能更加科学合理地节约成本、提升效率，并在满足读者需求的同时促进销售，挖掘更多潜在利润。该产品定价模型也可以用于其他领域的产品定价，如金融产品的定价。

## 一、模型搭建

### 1、读取数据

In [9]:
import pandas as pd
df = pd.read_excel('datasets/产品定价模型.xlsx')
df.head()

,页数,类别,彩印,纸张,价格
0,207,技术类,0,双胶纸,60
1,210,技术类,0,双胶纸,62
2,206,技术类,0,双胶纸,62
3,218,技术类,0,双胶纸,64
4,209,技术类,0,双胶纸,60


In [14]:
# 用value_counts()函数分类统计图书的类别
df['类别'].value_counts()

技术类    336
教辅类    333
办公类    331
Name: 类别, dtype: int64

In [13]:
# 统计图书的纸张类型
df['纸张'].value_counts()

双胶纸    615
铜版纸    196
书写纸    189
Name: 纸张, dtype: int64

### 2．类型文本变量的处理
因为“类别”和“纸张”两列是分类型文本变量，所以可以用LabelEncoder()函数进行数值化处理，便于后续进行模型拟合。

In [15]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['类别'] = le.fit_transform(df['类别'])  # 数值化处理类别

df['类别'].value_counts()

1    336
2    333
0    331
Name: 类别, dtype: int64

In [16]:
# 也可以用pandas库中的replace()函数进行替换
df['类别'] = df['类别'].replace({'办公类': 0, '技术类': 1, '教辅类':  2})

In [17]:
le = LabelEncoder()
df['纸张'] = le.fit_transform(df['纸张'])

df['纸张'].value_counts()

1    615
2    196
0    189
Name: 纸张, dtype: int64

In [18]:
df.head()

,页数,类别,彩印,纸张,价格
0,207,1,0,1,60
1,210,1,0,1,62
2,206,1,0,1,62
3,218,1,0,1,64
4,209,1,0,1,60


### 3、提取特征变量和目标变量

In [19]:
x = df.drop(columns='价格')
y = df['价格']

### 4、划分测试集和训练集

In [20]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123) # 设置train_test_split()函数的test_size参数为0.2，即测试集数据占20%；设置random_state参数为123（此数字无特殊含义，可以换成其他数字），使得每次运行代码划分数据的结果保持一致。

### 5、模型训练及搭建

In [21]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(random_state=123)
model.fit(x_train, y_train)

GradientBoostingRegressor(random_state=123)

## 二、模型预测及评估

### 1、对测试集数据进行预测

In [22]:
y_pred = model.predict(x_test)
y_pred[0:50]

array([ 71.15004038,  79.56199921,  68.21751792,  90.78788507,
        78.88479128,  42.28022702,  39.27334177,  60.74670841,
        53.59744659,  77.65931771,  80.22295545,  76.04437155,
        79.56199921,  58.40372895,  79.65245266,  44.27997693,
        53.18177447,  35.31452467,  92.1798291 ,  58.40372895,
        41.96644278,  99.50466356,  80.22295545,  79.69648341,
        91.45061741,  42.93885741,  42.86973046,  75.71824996,
        48.55203652,  62.94185778,  39.47077874,  61.54190648,
        95.18389309,  51.88118394,  65.1293139 ,  50.17577837,
        39.54495179,  83.63542315,  56.24632221, 102.1176112 ,
        48.89080247,  49.23639342,  33.03502962,  52.74862135,
        35.47220867,  35.00370671,  53.9446399 ,  74.62364353,
        35.31452467,  53.9446399 ])

### 2、汇总预测值和实际值，进行对比

In [23]:
a = pd.DataFrame()
a['预测值'] = list(y_pred)
a['实际值'] = list(y_test)
a.head()

,预测值,实际值
0,71.150040,75
1,79.561999,84
2,68.217518,68
3,90.787885,90
4,78.884791,85


In [25]:
# 还可以用模型自带的score()函数查看模型的预测效果
score = model.score(x_test, y_test)
score # 模型准确度评分score为0.874，说明模型的预测效果不错,这个评分其实就是模型的R-squared值

0.8741691363311168

In [26]:
# 查看模型的R-squared值
from sklearn.metrics import r2_score
r2 = r2_score(y_test, model.predict(x_test))
r2

0.8741691363311168

### 3、查看各个特征变量的特征重要性

In [27]:
model.feature_importances_

array([0.49070203, 0.44718694, 0.04161545, 0.02049558])

In [28]:
features = x.columns
importances = model.feature_importances_

a = pd.DataFrame()
a['特征'] = features
a['特征重要性'] = importances
a = a.sort_values('特征重要性', ascending=False)
a

,特征,特征重要性
0,页数,0.490702
1,类别,0.447187
2,彩印,0.041615
3,纸张,0.020496


可以看到，重要性最高的是“页数”和“类别”，一般来说，页数越多的图书价格越高，技术类和办公类图书的价格略高于教辅类图书的价格。“彩印”和“纸张”的重要性较低，对图书价格的影响较小。

## 三、模型参数介绍

In [29]:
from sklearn.ensemble import GradientBoostingRegressor
GradientBoostingRegressor?